In [5]:
!pip install torchvision
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.metrics import precision_score, recall_score
from tqdm import tqdm
import torch
from transformers import AutoTokenizer, AutoModel
from torchvision import models, transforms
from PIL import Image

# Device Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Load and Preprocess Dataset
def load_data(csv_path, image_dir):
    data = pd.read_csv(csv_path)
    images = []
    captions = []
    labels = []

    for idx in range(len(data)):
        img_name = str(data.loc[idx, 'image_id'])
        if not img_name.endswith(".jpg"):
            img_name += ".jpg"
        img_path = os.path.join(image_dir, img_name)

        if os.path.exists(img_path):
            images.append(img_path)
            captions.append(data.loc[idx, 'transcriptions'])
            if 'labels' in data.columns:
                labels.append(int(data.loc[idx, 'labels']))

    return images, captions, labels

# Feature Extraction Functions
def extract_text_features(captions, tokenizer, text_model, max_len=128):
    text_features = []
    for caption in tqdm(captions, desc="Extracting Text Features"):
        inputs = tokenizer(
            caption, return_tensors="pt", truncation=True, padding="max_length", max_length=max_len
        ).to(device)
        with torch.no_grad():
            outputs = text_model(**inputs)
            text_features.append(outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy())
    return np.array(text_features)

def extract_image_features(image_paths, visual_model, transform):
    image_features = []
    for img_path in tqdm(image_paths, desc="Extracting Image Features"):
        image = Image.open(img_path).convert("RGB")
        image = transform(image).unsqueeze(0).to(device)
        with torch.no_grad():
            features = visual_model(image).squeeze().cpu().numpy()
        image_features.append(features)
    return np.array(image_features)

from sklearn.metrics import precision_score, recall_score

# Training and Evaluation
def train_and_evaluate(X_train, y_train, X_test, y_test, model, model_name):
    print(f"\nTraining {model_name}...")
    model.fit(X_train, y_train)
    preds = model.predict(X_test)

    # Compute metrics
    precision = precision_score(y_test, preds, average="macro")
    recall = recall_score(y_test, preds, average="macro")
    macro_f1 = f1_score(y_test, preds, average="macro")

    print(f"\nResults for {model_name}:")
    print(classification_report(y_test, preds))
    print("Accuracy:", accuracy_score(y_test, preds))
    print(f"Macro Precision: {precision:.4f}")
    print(f"Macro Recall: {recall:.4f}")
    print(f"Macro F1 Score: {macro_f1:.4f}")
    return model

def main():
    # Paths
    train_csv = "/kaggle/input/malayalam-labelled-dataset/Dataset with label/train/train.csv"
    train_images = "/kaggle/input/malayalam-labelled-dataset/Dataset with label/train/images"
    test_csv = "/kaggle/input/malayalam-labelled-dataset/Dataset with label/test/test_with_labels.csv"
    test_images = "/kaggle/input/malayalam-labelled-dataset/Dataset with label/test/images"

    # Load Data
    train_images, train_captions, train_labels = load_data(train_csv, train_images)
    test_images, test_captions, test_labels = load_data(test_csv, test_images)

    # Tokenizer and Models
    tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
    text_model = AutoModel.from_pretrained("ai4bharat/indic-bert").to(device)
    visual_model = models.resnet50(pretrained=True)
    visual_model = torch.nn.Sequential(*list(visual_model.children())[:-1]).to(device)

    # Transform for Images
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # Feature Extraction
    print("\nExtracting Text Features...")
    train_text_features = extract_text_features(train_captions, tokenizer, text_model)
    test_text_features = extract_text_features(test_captions, tokenizer, text_model)

    print("\nExtracting Image Features...")
    train_image_features = extract_image_features(train_images, visual_model, transform)
    test_image_features = extract_image_features(test_images, visual_model, transform)

    # Combine Features
    X_train = np.concatenate([train_text_features, train_image_features], axis=1)
    X_test = np.concatenate([test_text_features, test_image_features], axis=1)
    y_train = np.array(train_labels)
    y_test = np.array(test_labels)

    # Define Models
    classifiers = {
        "Logistic Regression": LogisticRegression(max_iter=1000),
        "Support Vector Machine": SVC(kernel="linear", probability=True),
        "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42)
    }

    # Train and Evaluate Models
    for model_name, model in classifiers.items():
        train_and_evaluate(X_train, y_train, X_test, y_test, model, model_name)

if __name__ == "__main__":
    main()



Using device: cuda


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



Extracting Text Features...


Extracting Text Features: 100%|██████████| 200/200 [00:01<00:00, 123.05it/s]



Extracting Image Features...


Extracting Image Features: 100%|██████████| 200/200 [00:05<00:00, 34.80it/s]



Training Logistic Regression...

Results for Logistic Regression:
              precision    recall  f1-score   support

           0       0.80      0.77      0.78       122
           1       0.66      0.69      0.68        78

    accuracy                           0.74       200
   macro avg       0.73      0.73      0.73       200
weighted avg       0.74      0.74      0.74       200

Accuracy: 0.74
Macro Precision: 0.7276
Macro Recall: 0.7314
Macro F1 Score: 0.7292

Training Support Vector Machine...

Results for Support Vector Machine:
              precision    recall  f1-score   support

           0       0.80      0.80      0.80       122
           1       0.68      0.68      0.68        78

    accuracy                           0.75       200
   macro avg       0.74      0.74      0.74       200
weighted avg       0.75      0.75      0.75       200

Accuracy: 0.75
Macro Precision: 0.7373
Macro Recall: 0.7373
Macro F1 Score: 0.7373

Training Random Forest...

Results for 

In [6]:
# Install required libraries
!pip install torchvision transformers

# Import libraries
import os
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from torchvision import models, transforms
from PIL import Image

# Device Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Paths
train_csv_path = "/kaggle/input/malayalam-labelled-dataset/Dataset with label/train/train.csv"
train_images_path = "/kaggle/input/malayalam-labelled-dataset/Dataset with label/train/images"
dev_csv_path = "/kaggle/input/malayalam-labelled-dataset/Dataset with label/dev/dev.csv"
dev_images_path = "/kaggle/input/malayalam-labelled-dataset/Dataset with label/dev/images"
test_csv_path = "/kaggle/input/malayalam-labelled-dataset/Dataset with label/test/test_with_labels.csv"
test_images_path = "/kaggle/input/malayalam-labelled-dataset/Dataset with label/test/images"

# Load and Preprocess Dataset
def load_data(csv_path, image_dir):
    data = pd.read_csv(csv_path)
    images = []
    captions = []
    labels = []

    for idx in range(len(data)):
        img_name = str(data.loc[idx, 'image_id'])
        if not img_name.endswith(".jpg"):
            img_name += ".jpg"
        img_path = os.path.join(image_dir, img_name)

        if os.path.exists(img_path):
            images.append(img_path)
            captions.append(data.loc[idx, 'transcriptions'])
            if 'labels' in data.columns:
                labels.append(int(data.loc[idx, 'labels']))

    return images, captions, labels

# Feature Extraction Functions
def extract_text_features(captions, tokenizer, text_model, max_len=128):
    text_features = []
    for caption in tqdm(captions, desc="Extracting Text Features"):
        inputs = tokenizer(
            caption, return_tensors="pt", truncation=True, padding="max_length", max_length=max_len
        ).to(device)
        with torch.no_grad():
            outputs = text_model(**inputs)
            text_features.append(outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy())
    return np.array(text_features)

def extract_image_features(image_paths, visual_model, transform):
    image_features = []
    visual_model.eval()  # Ensure the model is in evaluation mode
    for img_path in tqdm(image_paths, desc="Extracting Image Features"):
        image = Image.open(img_path).convert("RGB")
        image = transform(image).unsqueeze(0).to(device)
        with torch.no_grad():
            features = visual_model(image).squeeze().cpu().numpy()
        image_features.append(features)
    return np.array(image_features)

# Evaluation Function
def evaluate_model(y_true, y_pred, model_name):
    print(f"\nResults for {model_name}:")
    print(classification_report(y_true, y_pred))
    print("Accuracy:", accuracy_score(y_true, y_pred))
    print("Macro Precision:", precision_score(y_true, y_pred, average="macro"))
    print("Macro Recall:", recall_score(y_true, y_pred, average="macro"))
    print("Macro F1 Score:", f1_score(y_true, y_pred, average="macro"))

# CNN Model
class CNNModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.fc1 = nn.Linear(128 * (input_dim // 2), 64)
        self.fc2 = nn.Linear(64, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add a dummy channel dimension: (batch_size, 1, input_dim)
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# BiLSTM Model
class BiLSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(BiLSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add a dummy sequence dimension: (batch_size, 1, input_dim)
        h_lstm, _ = self.lstm(x)
        out = self.fc(h_lstm[:, -1, :])  # Use the last hidden state
        return out

# BiLSTM + CNN Model
class BiLSTMCNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(BiLSTMCNNModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, bidirectional=True, batch_first=True)
        self.conv1 = nn.Conv1d(in_channels=hidden_dim * 2, out_channels=128, kernel_size=3, padding=1)
        self.fc = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # Debugging: Print input shape
        print(f"Input shape: {x.shape}")

        if len(x.shape) == 2:  # (batch_size, input_dim)
            x = x.unsqueeze(1)  # (batch_size, 1, input_dim)

        # LSTM processing
        h_lstm, _ = self.lstm(x)  # (batch_size, sequence_length, hidden_dim * 2)
        print(f"LSTM output shape: {h_lstm.shape}")

        h_lstm = h_lstm.permute(0, 2, 1)  # (batch_size, hidden_dim * 2, sequence_length)
        print(f"Permuted LSTM output shape: {h_lstm.shape}")

        # Convolution
        h_conv = nn.functional.relu(self.conv1(h_lstm))  # (batch_size, 128, sequence_length)
        print(f"Conv1d output shape: {h_conv.shape}")

        # Global average pooling
        h_pool = torch.mean(h_conv, dim=2)  # (batch_size, 128)
        print(f"Global average pooling output shape: {h_pool.shape}")

        # Fully connected layer
        out = self.dropout(self.fc(h_pool))  # (batch_size, num_classes)
        print(f"Final output shape: {out.shape}")

        return out
        
# Training and Evaluation Function for CNN
def train_and_evaluate_cnn(X_train, y_train, X_dev, y_dev, X_test, y_test, input_dim, num_classes, epochs=10, batch_size=32):
    model = CNNModel(input_dim, num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

    # Convert data to PyTorch tensors
    X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train = torch.tensor(y_train, dtype=torch.long).to(device)
    X_dev = torch.tensor(X_dev, dtype=torch.float32).to(device)
    y_dev = torch.tensor(y_dev, dtype=torch.long).to(device)
    X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_test = torch.tensor(y_test, dtype=torch.long).to(device)

    # DataLoader
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training
    for epoch in range(epochs):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        scheduler.step()

        # Validation
        model.eval()
        with torch.no_grad():
            outputs = model(X_dev)
            _, preds = torch.max(outputs, 1)
            val_loss = criterion(outputs, y_dev)
            print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}, Val Loss: {val_loss.item()}")

    # Evaluation on Test Data
    model.eval()
    with torch.no_grad():
        outputs = model(X_test)
        _, preds = torch.max(outputs, 1)
        evaluate_model(y_test.cpu(), preds.cpu(), "CNN")

# Training and Evaluation Function for BiLSTM
def train_and_evaluate_bilstm(X_train, y_train, X_dev, y_dev, X_test, y_test, input_dim, hidden_dim, num_classes, epochs=10, batch_size=32):
    model = BiLSTMModel(input_dim, hidden_dim, num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

    # Convert data to PyTorch tensors
    X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train = torch.tensor(y_train, dtype=torch.long).to(device)
    X_dev = torch.tensor(X_dev, dtype=torch.float32).to(device)
    y_dev = torch.tensor(y_dev, dtype=torch.long).to(device)
    X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_test = torch.tensor(y_test, dtype=torch.long).to(device)

    # DataLoader
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training
    for epoch in range(epochs):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        scheduler.step()

        # Validation
        model.eval()
        with torch.no_grad():
            outputs = model(X_dev)
            _, preds = torch.max(outputs, 1)
            val_loss = criterion(outputs, y_dev)
            print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}, Val Loss: {val_loss.item()}")

    # Evaluation on Test Data
    model.eval()
    with torch.no_grad():
        outputs = model(X_test)
        _, preds = torch.max(outputs, 1)
        evaluate_model(y_test.cpu(), preds.cpu(), "BiLSTM")

# Training and Evaluation Function for BiLSTM + CNN
def train_and_evaluate_bilstm_cnn(X_train, y_train, X_dev, y_dev, X_test, y_test, input_dim, hidden_dim, num_classes, epochs=10, batch_size=32):
    model = BiLSTMCNNModel(input_dim, hidden_dim, num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

    # Convert data to PyTorch tensors
    X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train = torch.tensor(y_train, dtype=torch.long).to(device)
    X_dev = torch.tensor(X_dev, dtype=torch.float32).to(device)
    y_dev = torch.tensor(y_dev, dtype=torch.long).to(device)
    X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_test = torch.tensor(y_test, dtype=torch.long).to(device)

    # DataLoader
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training
    for epoch in range(epochs):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        scheduler.step()

        # Validation
        model.eval()
        with torch.no_grad():
            outputs = model(X_dev)
            _, preds = torch.max(outputs, 1)
            val_loss = criterion(outputs, y_dev)
            print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}, Val Loss: {val_loss.item()}")

    # Evaluation on Test Data
    model.eval()
    with torch.no_grad():
        outputs = model(X_test)
        _, preds = torch.max(outputs, 1)
        evaluate_model(y_test.cpu(), preds.cpu(), "BiLSTM + CNN")
        
# Main Function

def main():
    # Load Data
    train_image_paths, train_captions, train_labels = load_data(train_csv_path, train_images_path)
    dev_image_paths, dev_captions, dev_labels = load_data(dev_csv_path, dev_images_path)
    test_image_paths, test_captions, test_labels = load_data(test_csv_path, test_images_path)

    # Define Tokenizer and Text Model
    tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
    text_model = AutoModel.from_pretrained("bert-base-multilingual-cased").to(device)

    # Define Visual Model
    visual_model = models.resnet50(pretrained=True)
    visual_model.fc = nn.Identity()  # Use the penultimate layer as the feature extractor
    visual_model = visual_model.to(device)

    # Define Image Transform
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # Extract Features
    print("Extracting Text Features...")
    train_text_features = extract_text_features(train_captions, tokenizer, text_model)
    dev_text_features = extract_text_features(dev_captions, tokenizer, text_model)
    test_text_features = extract_text_features(test_captions, tokenizer, text_model)

    print("Extracting Image Features...")
    train_image_features = extract_image_features(train_image_paths, visual_model, transform)
    dev_image_features = extract_image_features(dev_image_paths, visual_model, transform)
    test_image_features = extract_image_features(test_image_paths, visual_model, transform)

    # Combine Features
    train_features = np.hstack((train_text_features, train_image_features))
    dev_features = np.hstack((dev_text_features, dev_image_features))
    test_features = np.hstack((test_text_features, test_image_features))

    # Train and Evaluate Models
    input_dim = train_features.shape[1]
    num_classes = len(set(train_labels))
    hidden_dim = 128  # For BiLSTM

    print("\nTraining CNN Model...")
    train_and_evaluate_cnn(train_features, train_labels, dev_features, dev_labels, test_features, test_labels, input_dim, num_classes)

    print("\nTraining BiLSTM Model...")
    train_and_evaluate_bilstm(train_features, train_labels, dev_features, dev_labels, test_features, test_labels, input_dim, hidden_dim, num_classes)

    print("\nTraining BiLSTM + CNN Model...")
    train_and_evaluate_bilstm_cnn(train_features, train_labels, dev_features, dev_labels, test_features, test_labels, input_dim, hidden_dim, num_classes)

if __name__ == "__main__":
    main()


Using device: cuda


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

...

Extracting Image Features...


Extracting Image Features: 100%|██████████| 200/200 [00:05<00:00, 38.21it/s]



Training CNN Model...
Epoch 1/10, Loss: 0.687190055847168, Val Loss: 0.6525322198867798
Epoch 2/10, Loss: 0.7020477652549744, Val Loss: 0.6450766324996948
Epoch 3/10, Loss: 0.7403168082237244, Val Loss: 0.6063914895057678
Epoch 4/10, Loss: 0.6555379033088684, Val Loss: 0.6180673241615295
Epoch 5/10, Loss: 0.6114636063575745, Val Loss: 0.5684558153152466
Epoch 6/10, Loss: 0.6048188805580139, Val Loss: 0.5832468867301941
Epoch 7/10, Loss: 0.6645528078079224, Val Loss: 0.5992092490196228
Epoch 8/10, Loss: 0.67783522605896, Val Loss: 0.5824341773986816
Epoch 9/10, Loss: 0.6288920640945435, Val Loss: 0.5877923369407654
Epoch 10/10, Loss: 0.6097722053527832, Val Loss: 0.5810610055923462

Results for CNN:
              precision    recall  f1-score   support

           0       0.61      1.00      0.76       122
           1       0.00      0.00      0.00        78

    accuracy                           0.61       200
   macro avg       0.30      0.50      0.38       200
weighted avg       

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

Epoch 1/10, Loss: 0.47954320907592773, Val Loss: 0.4826173186302185
Epoch 2/10, Loss: 0.353268563747406, Val Loss: 0.5374100208282471
Epoch 3/10, Loss: 0.23191724717617035, Val Loss: 0.506673276424408
Epoch 4/10, Loss: 0.33520039916038513, Val Loss: 0.6103768348693848
Epoch 5/10, Loss: 0.1951366364955902, Val Loss: 0.5820045471191406
Epoch 6/10, Loss: 0.2093452513217926, Val Loss: 0.4393930435180664
Epoch 7/10, Loss: 0.16213460266590118, Val Loss: 0.4827810227870941
Epoch 8/10, Loss: 0.17968909442424774, Val Loss: 0.48018330335617065
Epoch 9/10, Loss: 0.19861088693141937, Val Loss: 0.4990668296813965
Epoch 10/10, Loss: 0.2400296926498413, Val Loss: 0.4801900386810303

Results for BiLSTM:
              precision    recall  f1-score   support

           0       0.83      0.87      0.85       122
           1       0.78      0.73      0.75        78

    accuracy                           0.81       200
   macro avg       0.81      0.80      0.80       200
weighted avg       0.81      0.